## Mit Modulen Zwecke modellieren

Software leistet für den Anwender vor allem mittels Logik. Wächst Logik über wenige Zeilen hinaus, sinkt ihre Verständlichkeit und Testbarkeit rapide. Darüber hinaus leistet "Logik auf einem Haufen" Wiederholungen Vorschub, die zu doppelter Arbeit und Inkonsistenzen führen.

Diesen negativen Effekten wirkt die Gruppierung von Logik in Funktionen entgegen. Funktionen fassen Logik zu verhaltenserzeugenden Einheiten auf höheren Abstraktionsniveau zusammen. Operationen tun das in direkt und in reiner Form: sie enthalten ausschließlich Logik. Integrationen hingegen tun das indirekt durch die Zusammenfassung von Operationen oder anderen Integrationen.

Die direkte oder indirekte Gruppierung von Logik nennt Flow-Design Komposition. In einer Komposition wird bewusst Verschiedenartiges zusammengestellt zu etwas Neuartigem.

/// komposition: komplementäre puzzleteile zu einer neuen figur zusammensetzen (tangram) (mod1)

Wenn es nur Addition, Vergleich und eine Schleife gibt, dann kann aus diesen Logik-Bausteinen der "neue" (vereinfachte) mathematische Operator Multiplikation komponiert werden:

In [3]:
int Multiply(int a, int b) {
    if (a == 0) return 0;
    
    var result = 0;
    while(a > 0) {
        result += b;
        a += -1;
    }
    return result;
}

display(Multiply(2,3));
display(Multiply(0,3));

6

0

Einfache Kompositionen lassen sich zu komplizierteren zusammensetzen, aus denen wiederum kompliziertere komponiert werden können usw. usf. So kann das Verhalten von Software Ebene für Ebene einfacher und einfacher zugänglich gemacht werden.

Komposition ist die Abstraktion, mit der Logik verwendbar, gar wiederverwendbar, und testbar und einmalig gemacht wird.

Viele kleine Komposite (Funktionen) sind eine gute Sache als feines Granulat für den Bau von Verhalten. Allerdings werfen sie am Ende auch ein Problem durch wachsende Masse auf: man verliert alsbald die Übersicht.

Wachsende Logik "auf einem Haufen" hat bei allen Nachteilen zumindest noch für sich, dass man sie von oben nach unten lesend deuten kann. Die "Geschichte", wie sie Verhalten herstellt, wird linear erzählt. Insofern ist Logik sogar weniger "Haufen" als "Liste".

Funktionen "auf einem Haufen" stehen hingegen in keinem solch simplen Zusammenhang. Von außen betrachtet sind sie ohne Ordnung. In der einen oder anderen Programmiersprache mag zwar eine gewisse Reihenfolge erzwungen werden, als dass eine Definition der Nutzung vorhergehen muss. Doch auch das macht am Ende keinen großen Unterschied.

Ohne weitere Maßnahmen bilden Funktionen "einen Haufen", in dem alle Elemente mit allen verbunden sein könn(t)en. Und das ist ein Rezept für Chaos. Ordnung sieht anders aus. Ordnung braucht Unterscheidung, sie braucht Grenzen.

Funktionen ziehen auch Grenzen in die Logik ein. Die dienen primär der Herstellung von Verhaltenseinheiten im Sinne von Schritten in einem Produktionsprozess.

Was es nun aber braucht, um eine wachsende Zahl von Funktionen im Griff zu behalten, das sind Grenzen der Verständlichkeit. Es braucht Geltungsbereiche (scope), damit nicht alles überall gleich relevant erscheint.

### Geltungsbereiche abgrenzen mit Modulen

Module sind das Mittel in Programmiersprachen, um Geltungsbereiche abzugrenzen. Es sind Container zur Sammlung von Funktionen. Das Kriterium für die Gruppierung ist dabei jedoch nicht Verschiedenheit, sondern Ähnlichkeit. Die Komposition lebt davon, dass Verschiedenes zu etwas Neuem zusammengefasst wird. Module hingegen stehen für Aggregation und die lebt davon, dass Ähnliches unter einem gemeinsamem Zweck zusammengefasst wird.

/// aggregation: verschiedene komplementäre formen nach farbe od einem anderen kriterium gruppieren (2)

Zunächst sind da nur Funktionen als Kompositionen:

In [23]:
string[] SplitIntoWords(string text) => text.Split(new[]{' ', '\t', '\n', '\r'}, StringSplitOptions.RemoveEmptyEntries);

IEnumerable<(string filename, string text)> LoadTexts(string path)
    => System.IO.Directory.GetFiles(path, "*.txt").Select(x => (x, System.IO.File.ReadAllText(x)));

int CountWords(IEnumerable<string> words) => words.Count(x => x.Length > 3);

void SaveAnalysis(string filename, IEnumerable<(string filename, int numberOfWords)> analysis) {
    using(var sw = new System.IO.StreamWriter(filename, false)) {
        foreach(var a in analysis)
            sw.WriteLine($"{a.filename},{a.numberOfWords}");
    }
}


display(SplitIntoWords("the quick brown fox"));

display(LoadTexts("samples"));

display(CountWords(new[]{"the", "quick", "brown", "fox"}));

SaveAnalysis("samples/analysis.csv", new[]{("foo.txt", 1), ("bar.txt", 41)});
display(System.IO.File.ReadAllText("samples/analysis.csv"));

index,value
0,the
1,quick
2,brown
3,fox


index,Item1,Item2
0,samples/poem.txt,Humpty Dumpty sat on the wall Humpty Dumpty had a great fall All the king’s horses and all the king’s men Couldn’t put Humpty together again Humpty Dumpty sat on the wall Humpty Dumpty had a great fall All the king’s horses and all the king’s men Couldn’t put Humpty together again Humpty Dumpty sat on the wall Humpty Dumpty had a great fall All the king’s horses and all the king’s men Couldn’t put Humpty together again 1870
1,samples/shakespeare.txt,"She should have died hereafter; There would have been a time for such a word. To-morrow, and to-morrow, and to-morrow, Creeps in this petty pace from day to day To the last syllable of recorded time, And all our yesterdays have lighted fools The way to dusty death. Out, out, brief candle! Life's but a walking shadow, a poor player That struts and frets his hour upon the stage And then is heard no more: it is a tale Told by an idiot, full of sound and fury, Signifying nothing."


2

foo.txt,1
bar.txt,41


Mit diesen Funktionen kann eine umfassendere komponiert werden, in der sie zu einem Prozess zusammengefügt sind, der ein gewünschtes Gesamtverhalten herstellt. (Dass dabei noch ein wenig _glue logic_ hinzukommt, sei an dieser Stelle vernachlässigt; die folgende Integration ist also nicht 100% sauber.) 

In [28]:
int AnalyzeFiles(string path) {
    var content = LoadTexts(path);
    var parsed = content.Select(x => (x.filename, words:SplitIntoWords(x.text)));
    var analyzed = parsed.Select(x => (x.filename, numberOfWords:CountWords(x.words)));
    SaveAnalysis(System.IO.Path.Combine(path, "analysis.csv"), analyzed);
    return content.Count();
}


display(AnalyzeFiles("samples"));

2

Überblick über die Funktionen entsteht durch die Komposition jedoch nicht. Wären es mehr als die vier, müsste einiger Aufwand getrieben werden, um die "Bausteinvorrat" erstmal zu sichten.

Dem kann mit Modulen abgeholfen werden. Mehr Übersichtlichkeit entsteht, wenn Funktionen "thematisch" gruppiert vorliegen. Eine solche Gruppierung könnte z.B. so aussehen:

In [34]:
class FileAccess {
    public static IEnumerable<(string filename, string text)> LoadTexts(string path)
        => System.IO.Directory.GetFiles(path, "*.txt").Select(x => (x, System.IO.File.ReadAllText(x)));
    
    public static void SaveAnalysis(string filename, IEnumerable<(string filename, int numberOfWords)> analysis) {
        using(var sw = new System.IO.StreamWriter(filename, false)) {
            foreach(var a in analysis)
                sw.WriteLine($"{a.filename},{a.numberOfWords}");
        }
    }
}

    
class Domain {
    public static IEnumerable<(string filename, int numberOfWords)> Analyze(IEnumerable<(string filename, string text)> files) {
            var parsed = files.Select(x => (x.filename, words:SplitIntoWords(x.text)));
            return parsed.Select(x => (x.filename, numberOfWords:CountWords(x.words)));
    }

    private static string[] SplitIntoWords(string text) => text.Split(new[]{' ', '\t', '\n', '\r'}, StringSplitOptions.RemoveEmptyEntries);

    private static int CountWords(IEnumerable<string> words) => words.Count(x => x.Length > 3);
}


class FileAnalysis {
    public static int Analyze(string path) {
        var content = FileAccess.LoadTexts(path);
        var result = Domain.Analyze(content);
        FileAccess.SaveAnalysis(System.IO.Path.Combine(path, "analysis.csv"), result);
        return content.Count();
    }
}


display(FileAnalysis.Analyze("samples"));

2

Aus 5 Funktionen sind 3 Module geworden. Auch wenn diese Reduktion der Strukturelemente noch nicht aufsehenerregend ist, zeigt sie in die richtige Richtung. 40% weniger Teile machen die Lösung übersichtlicher.

/// zuerst 4+1 funktionen, dann 3 module mit (4+1)+1 funktionen; sichtbarkeit andeuten durch positionierung auf dem rand des moduls (3)

Die erste Leistung der Module besteht in der _physischen Klammerung_. Die geschieht hier mit programmiersprachlichen Mitteln, also syntaktisch. Das ist in Sprachen wie Java oder C# nötig, in denen es keine "freien" Funktionen gibt. In anderen Sprachen, z.B. Phython oder Kotlin, kann jedoch auch durch schiere Zusammenfassung von Funktionen in derselben Datei geklammert werden.

Die zweite Leistung der Module besteht in der _Beschränkung des Geltungsbereiches_. Öffentliche (public) Funktionen der Module sind global gültig, private Funktionen hingegen nur innerhalb ihres Moduls. Davon macht `Domain` Gebrauch, um zwei Funktionen "zu verstecken". Stattdessen veröffentlicht das Modul eine Komposition beider, um es ihren Nutzern einfacher zu machen, eine Analyse durchzuführen.

Module sind also nicht nur passive Sammelkörbe, sondern steuern den Detailreichtum der Funktionslandschaft aktiv über Sichtbarkeitsattribute. So definieren sie eine _Schnittstelle_ für den Zweck, den sie repräsentieren.

#### Zweckmäßige Zusammenfassungen

Worin die Gemeinsamkeit von Funktionen besteht, die in einem Modul aggregiert werden, das liegt letztlich im Auge des Entwicklers. Eine Zusammenfassung in einem Modul ist für ihn Ausdruck eines gewissen Zwecks, den sie gemeinsam verfolgen.

Auch wenn die Bestimmung eines solchen Zwecks recht subjektiv sein und zu Diskussionen führen kann, kennt Flow-Design einige "harte" Kriterien, die als Ausgangspunkt herangezogen werden können:

* Funktionen nutzen denselben API, d.h. sie greifen auf dieselbe (Art von) Ressource zu.
* Funktionen haben gemeinsamen in-memory Zustand.
* Funktionen arbeiten auf denselben Input-Daten.
* Funktionen sind relevant für denselben Stakeholder.

Ein Beispiel für das erste Kriterium sind die Funktionen, die in `FileAccess` zusammengefasst wurden. Beide greifen auf das Dateisystem zu. Die eine lesend, die andere schreibend, aber sie benutzen denselben API (repräsentiert durch den .NET CLR Namespace `System.IO`).

Ein Beispiel für das dritte Kriterium sind die Funktionen in `Domain`. Beide arbeiten auf Input, der Dateiinhalte repräsentiert, die zu transformieren sind. Zuerst mit einem map-, dann mit einem reduce-Schritt.

Funktionen sind Antworten auf eine Wie-Frage: Wie wird ein Verhalten hergestellt?

Module sind Antworten auf die Was-Frage: Was für grundsätzliche "Verhaltensbereiche", d.h. Zwecke gibt es überhaupt?

Selbstverständlich profitieren auch Module bei der Beantwortung dieser Frage von Fokus. Je fokussierter, je klarer Umrissen der Zweck, zu dem Funktionen in ihnen zusammenkommen, desto leichter ist zu verstehen, was ein Modul bietet.

Während Funktionen für Entscheidungen zur Laufzeit stehen, die das Verhalten beeinflussen, stehen Module für Entscheidungen zur Entwicklungszeit; Entscheidungen des Kunden, d.h. Festlegungen zu Aspekten der Anforderungen, und Entwurfsentscheidungen der Entwickler können mit Modulen besonders ausdruckskräftig repräsentiert werden. Das Modul `FileAccess` steht z.B. für die grundlegende Entscheidung, dass die Analyse auf dem Dateisystem abläuft und nicht mit Texten arbeitet, die der Benutzer eingibt.

#### Abhängigkeiten zwischen Modulen

Funktionen sind von einander abhängig, wenn die eine die andere benutzt. Das ist unumgänglich, sollte allerdings aus Sicht von Flow-Design durch Einhaltung des IOSP unter Kontrolle gehalten werden.

/// abhängigkeit der funktionen aus dem obigen beispiel (4)

Da Module Funktionen aggregieren, stehen Module indirekt auch in Abhängigkeitsverhältnissen. Auch das ist unumgänglich.

/// abhängigkeit der module aus dem beispiel (5)

IOSP für module

#### Datenstrukturen

sind keine module

* Namensraum
  keine instanziierung
  nicht in allen sprachen
* Klassen
* Bibliotheken
* Pakete
* Komponenten
* Services
* Module in der Softwarezelle
* Module im Softwareuniversum